# Access locations and times of Veery encounters

For this challenge, you will use a database called the [Global
Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF
is compiled from species observation data all over the world, and
includes everything from museum specimens to photos taken by citizen
scientists in their backyards.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Explore GBIF</div></div><div class="callout-body-container callout-body"><p>Before your get started, go to the <a
href="https://www.gbif.org/occurrence/search">GBIF occurrences search
page</a> and explore the data.</p></div></div>

> **Contribute to open data**
>
> You can get your own observations added to GBIF using
> [iNaturalist](https://www.inaturalist.org/)!

### Set up your code to prepare for download

We will be getting data from a source called [GBIF (Global Biodiversity
Information Facility)](https://www.gbif.org/). We need a package called
`pygbif` to access the data, which may not be included in your
environment. Install it by running the cell below:

In [16]:
%%bash
pip install pygbif

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>In the imports cell, we’ve included some packages that you will need.
Add imports for packages that will help you:</p>
<ol type="1">
<li>Work with reproducible file paths</li>
<li>Work with tabular data</li>
</ol></div></div>

In [17]:
import pandas as pd
import os
import pathlib

import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species

In [18]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'blue-jay-migration',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data, create directory
gbif_dir = os.path.join(data_dir, 'blue-jay-gbif')

os.makedirs(gbif_dir, exist_ok=True)

gbif_path = os.path.join(gbif_dir, 'blue-jay.csv')

In [19]:
%%bash
find ~/earth-analytics/data/blue-jay-migration/

/home/jovyan/earth-analytics/data/blue-jay-migration/
/home/jovyan/earth-analytics/data/blue-jay-migration/blue-jay-gbif


:::

### Register and log in to GBIF

You will need a [GBIF account](https://www.gbif.org/) to complete this
challenge. You can use your GitHub account to authenticate with GBIF.
Then, run the following code to save your credentials on your computer.

> **Warning**
>
> Your email address **must** match the email you used to sign up for
> GBIF!

> **Tip**
>
> If you accidentally enter your credentials wrong, you can set
> `reset_credentials=True` instead of `reset_credentials=False`.

In [27]:
reset_credentials = False
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, 'username'),
    GBIF_PWD=(getpass, 'password'),
    GBIF_EMAIL=(input, 'email'),
)
for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

In [26]:
%%bash
echo $GBIF_USER

hannah.rieder


### Get the species key

> ** Your task**
>
> 1.  Replace the `species_name` with the name of the species you want
>     to look up
> 2.  Run the code to get the species key

In [28]:
# Query species
species_info = species.name_lookup('Cyanocitta cristata', rank='species')

# Get the first result
first_result = species_info['results'][0]
first_result

{'key': 168007494,
 'datasetKey': 'a6c6cead-b5ce-4a4e-8cf5-1542ba708dec',
 'nubKey': 2482593,
 'parentKey': 168007493,
 'parent': 'Cyanocitta',
 'kingdom': 'Animalia',
 'phylum': 'Chordata',
 'order': 'Passeriformes',
 'family': 'Corvidae',
 'genus': 'Cyanocitta',
 'species': 'Cyanocitta cristata',
 'kingdomKey': 167920784,
 'phylumKey': 168001715,
 'classKey': 168003099,
 'orderKey': 168005163,
 'familyKey': 168007364,
 'genusKey': 168007493,
 'speciesKey': 168007494,
 'scientificName': 'Cyanocitta cristata',
 'canonicalName': 'Cyanocitta cristata',
 'nameType': 'SCIENTIFIC',
 'taxonomicStatus': 'ACCEPTED',
 'rank': 'SPECIES',
 'origin': 'SOURCE',
 'numDescendants': 0,
 'numOccurrences': 0,
 'habitats': [],
 'nomenclaturalStatus': [],
 'threatStatuses': [],
 'descriptions': [],
 'vernacularNames': [{'vernacularName': 'blåskrike', 'language': 'nob'}],
 'higherClassificationMap': {'167920784': 'Animalia',
  '168001715': 'Chordata',
  '168003099': 'Aves',
  '168005163': 'Passeriformes',


In [29]:
# Get the species key (nubKey)
species_key = first_result['nubKey']
species_key

2482593

In [30]:
# Check the result
first_result['species'], species_key

('Cyanocitta cristata', 2482593)

### Download data from GBIF

::: {.callout-task title=“Submit a request to GBIF”

1.  Replace `csv_file_pattern` with a string that will match **any**
    `.csv` file when used in the `glob` function. HINT: the character
    `*` represents any number of any values except the file separator
    (e.g. `/`)

2.  Add parameters to the GBIF download function, `occ.download()` to
    limit your query to:

    -   observations
    -   from 2023
    -   with spatial coordinates.

3.  Then, run the download. **This can take a few minutes**. :::

In [32]:

os.environ['GBIF_DOWNLOAD_KEY'] = '0003394-241007104925546'


In [34]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, '*.csv')
if not glob(gbif_pattern):
    # Submit query to GBIF
    # Only download once
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        gbif_query = occ.download([
            "speciesKey = 2482593",
            "year = 2023",
            "hasCoordinate = TRUE",
        ])
        download_key = gbif_query[0]
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]
    else: 
        download_key = os.environ['GBIF_DOWNLOAD_KEY']

    # Wait for the download to build
    wait = occ.download_meta(download_key)['status']
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']
        time.sleep(5)

    # Download GBIF data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=data_dir)

    # Unzip GBIF data
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path
gbif_path = glob(gbif_pattern)[0]

INFO:Download file size: 304308872 bytes
INFO:On disk at /home/jovyan/earth-analytics/data/blue-jay-migration/0003394-241007104925546.zip


### Load the GBIF data into Python

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Load GBIF data</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Look at the beginning of the file you downloaded using the code
below. What do you think the <strong>delimiter</strong> is?</li>
<li>Run the following code cell. What happens?</li>
<li>Uncomment and modify the parameters of <code>pd.read_csv()</code>
below until your data loads successfully and you have only the columns
you want.</li>
</ol></div></div>

You can use the following code to look at the beginning of your file:

In [35]:
!head -n 2 $gbif_path 

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
4686102821	4fa7b334-ce0d-4e88-aaae-2e0c138d049e	URN:catalog:CLO:EBIRD:OBS1601518595	Animalia	Chordata	Aves	Passeriformes	Corvidae	Cyanocitta	Cyanocitta cristata		SPECIES	Cyanocitta cristata (Linnaeus, 1758)	Cyanocitta cristata		US	Hartman Reserve Nature Center	Iowa	PRESENT	1	e2e717bf-551a-4917-bdc9-4fa0f342c530	42.52484	-92.41017							2023-01-08	8	1	2023	248259

In [36]:
# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    #delimiter='',
    #index_col='',
    #usecols=[]
)
gbif_df.head()

ParserError: Error tokenizing data. C error: Expected 2 fields in line 6, saw 3
